<a href="https://colab.research.google.com/github/mszkdt/fine-tuning_lab/blob/main/Piper_FineTuning_0528.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# ２．学習済LLMのカスタマイズ：Fine Tuning（追加学習）






![001_What_is_LLM](https://raw.githubusercontent.com/mszkdt/fine-tuning_lab/main/001_What_is_LLM.png)



----

![002_LLM_model_size](https://raw.githubusercontent.com/mszkdt/fine-tuning_lab/main/002_LLM_model_size.png)


----

![003_Fine-tuning.png](https://raw.githubusercontent.com/mszkdt/fine-tuning_lab/main/003_Fine-tuning.png)



----






# 一般的なファインチューニング実施までの工程

**1. 環境、データの準備**

* ファイチューニング実行環境と、モデルの訓練に使用するデータを準備します。
* これには、データの収集、クリーニング、前処理などが含まれます。

**2. トークナイザーの準備**

* 次に、テキストデータをモデルが理解できる形式に変換するためのトークナイザーを準備します。
* トークナイザーは、テキストをトークン（単語や文字などの単位）に分割し、それぞれのトークンを一意の数値（トークンID）に変換します。

**3. モデルの選択**

* 事前訓練済みのモデルを選択します。
* このモデルは、大量のテキストデータで事前に訓練されており、一般的な言語理解能力を持っています。

**4. データのトークナイゼーション**

* 準備したデータをトークナイザーでトークナイズし、モデルが理解できる形式に変換します。

**5. ファインチューニングの設定、実施**

* 最後に、ファインチューニングの設定を行います。
* これには、訓練のパラメータ（例えば、学習率やエポック数など）の設定、訓練データの指定、モデルの保存先の指定などが含まれます。

----



# 今回のLab内容

* Google Colabにて、 サイバーエージェントの OpenCalm を使い、iDRACのURIを学習させる
* 学習前と学習後の実行結果を確認する

### 参考：OpenCalm（おーぷんかーむ）とは

サイバーエージェントが2023年5月11日に公開された日本語LLM

*   商用利用が可能
*   データソース：日本語のオープンデータ(Wikipedia, Common Crawl)
*   パラメータ数： S 1.6億、 M 4億、 L 8.3億、 calm2 70億
*   AI開発基盤にPowerEdge XE9680、 NVIDIA® H100 GPUを利用
*   公開先： https://huggingface.co/cyberagent
*   開発者： https://www.cyberagent.co.jp/way/list/detail/id=29497



# 環境準備


**実行前にFineTuning用データを、自身のGoogle Driveへダウンロードする**

"/content/drive/MyDrive/redfish_llm.csv"



**pipで以下ライブラリをインストール**

* datasets
  * 機械学習や自然言語処理のための大規模なデータセットを効率的にロード、前処理、探索するためのツール

* accelerate
  * 同じPyTorchのコードを任意の分散設定で実行できるようにし、大規模な訓練と推論をシンプルで効率的かつ適応可能にするツール

* transformer
  * トランスフォーマーモデル（BERT、GPT-2など）を簡単に利用できるようにするためのツール
  * [torch]の部分は、このライブラリをPyTorchとともに使用するためのオプション

In [ ]:

### GoogleDriveマウント（学習データの一時保管）
#
# auth.authenticate_user()

from google.colab import auth
from google.colab import drive
from google.colab import output

drive.mount('/content/drive')


In [ ]:
# 学習データのダウンロード

!curl https://raw.githubusercontent.com/mszkdt/fine-tuning_lab/main/redfish_llm.csv > /content/drive/MyDrive/redfish_llm.csv

In [ ]:
!pip install datasets
!pip install accelerate
!pip install transformers[torch]



**インストールしたライブラリから、以下をインポート**

* import pandas as pd
  * pandasというライブラリをインポートする。pandasは、Pythonでデータ分析を行うための強力なライブラリで、データフレームという形式でデータを操作することができます。（pdは、pandasを参照するための省略形）

* from datasets import Dataset, DatasetDict
  * この行は、datasetsライブラリからDatasetとDatasetDictというクラスをインポートしています。これらのクラスは、Hugging Face社が提供するdatasetsライブラリの一部で、機械学習のデータセットを効率的に操作するためのものです。

* import transformers
  * transformersというライブラリ全体をインポートしています。transformersライブラリは、Hugging Face社が提供するもので、トランスフォーマーモデルを簡単に利用できるようにするためのものです。

* import torch
  * torchというライブラリをインポートしています。torchは、PyTorchとも呼ばれ、深層学習のためのオープンソースライブラリです。PyTorchは、tensor計算（NumPyのような）に強力なGPU加速を提供し、深層学習モデルの構築と訓練を支援します。


In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
import transformers
import torch

import wandb
wandb.init(mode="disabled")


In [ ]:
# GPUが利用可能かどうかを確認
#
# 参考：https://note.com/npaka/n/n1aa6f8c973d0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

---

# ファインチューニング準備







In [ ]:
# CSVファイルから pandas ライブラリを使用して、追加学習データ（CSV）を読み込む

df = pd.read_csv("/content/drive/MyDrive/redfish_llm.csv")

In [ ]:
# 追加学習データ（CSV）内容の表示
df

In [ ]:
# ベースとなるLLMモデルの指定
#
# cyberagent/open-calm-small
# cyberagent/open-calm-medium
# cyberagent/open-calm-large
# https://huggingface.co/cyberagent

base_model = "cyberagent/open-calm-medium"

# トークナイザーのロード（自然言語構造化された形式に変換し、モデルが理解できるようにする初期処理のゲートキーパー）
# * transformersライブラリのAutoTokenizerクラスのfrom_pretrainedメソッドを使用して、事前学習済みのトークナイザーをロード
# * from_pretrainedメソッドは、指定したモデル名（ここではbase_model）に対応するトークナイザーをロード

tokenizer = transformers.AutoTokenizer.from_pretrained(base_model)

# 事前学習済モデルのロード

model = transformers.AutoModelForCausalLM.from_pretrained(base_model)

# 追加学習データ用の前処理
#
# * トークナイザーを使用してテキストデータをモデルが理解できる形式に変換する
#   テキストデータをトークン化（単語やサブワードに分割）し、それを数値のインデックスに変換する処理を行う。
#
# * "pt"はPyTorchのテンソル形式を意味し、トークナイザーはトークン化されたデータをPyTorchテンソルとして出力する。
#   これにより、出力されたテンソルはそのままPyTorchを使用して構築されたモデルで学習に使用することができる。

def preprocess_function(examples):
    return tokenizer(examples["input"], return_tensors="pt")
    # return tokenizer(examples["input"], padding="max_length", max_length=8, truncation=True, return_tensors="pt")
    # return tokenizer(examples["input"], padding=True, max_length=24, truncation=True, return_tensors="pt")


In [ ]:
# 追加学習データの加工
#
# * train_dataset = Dataset.from_pandas(df)
#   pandasのデータフレームdfをHugging FaceのDatasetオブジェクトに変換する。
#   Dataset.from_pandas(df)メソッドは、pandasのデータフレームを引数として受け取り、それをDatasetオブジェクトに変換する。
#   この結果、データフレームの各行がDatasetの各エントリに対応します。

# * data = DatasetDict({ "train": train_dataset, })
#   変換したDatasetオブジェクトをDatasetDictオブジェクトに格納しています。
#   DatasetDictは、複数のDatasetオブジェクトを一元管理するためのクラスで、各Datasetオブジェクトはキーを指定してアクセスできます。
#   この例では、train_datasetを"train"のキーで格納しています。

# * data = data.map(lambda samples: tokenizer(samples["output"]), batched=True)
#   data（DatasetDictオブジェクト）の各エントリに対してトークナイザーを適用しています。
#
#      lambda samples: tokenizer(samples["output"])は、各エントリの"output"フィールドをトークナイザーに渡す無名関数。
#      batched=Trueは、バッチ単位でトークナイゼーションを行うことを指定しています。
#      これにより、一度に複数のエントリを処理することができ、効率的にトークナイゼーションを行うことができます。


train_dataset = Dataset.from_pandas(df)

data = DatasetDict(
    {
        "train": train_dataset,
    }
)

data = data.map(lambda samples: tokenizer(samples["output"]), batched=True)

In [ ]:
# 前処理後のdataから学習データセット（"train"キー）の2番目のエントリを取得している。
# Pythonのインデックスは0から始まるため、[1]は2番目のエントリを指す。

data["train"][1]

In [ ]:
# Fine Tuningの設定
# *** pip XXXのエラーが消えない場合は、ランタイム >セッション再起動　で解決 ***

# Hugging FaceのTransformersライブラリを使用して、モデルのFine Tuning（微調整）を設定するためのものです。

# trainer = transformers.Trainer(...): ここでは、Trainerクラスのインスタンスを作成しています。Trainerクラスは、モデルの訓練や評価を行うためのクラス
#    model=model: ここでは、微調整を行うモデルを指定
#    train_dataset=data["train"]: ここでは、訓練データセットを指定
#    args=transformers.TrainingArguments(...): ここでは、訓練のパラメータを設定。TrainingArgumentsクラスは、訓練のパラメータを管理するためのクラス。
#        logging_steps=1        : ログを出力するステップ数を指定しています。この例では、各ステップごとにログが出力されます。
#        output_dir="./output"  : モデルや訓練の結果を保存するディレクトリを指定しています。
#        num_train_epochs = 5   : 訓練のエポック数を指定しています。この例では、5エポック訓練します。
#                                 (機械学習の訓練プロセスにおける一つの単位を指す。具体的には、訓練データ全体がモデルに一度通過することを1エポック)
#        warmup_steps = 5       : 学習率のウォームアップステップ数を指定しています。ウォームアップステップでは、学習率が徐々に上昇します。
#        weight_decay = 0.1     : 重み減衰の値を指定しています。重み減衰は、過学習を防ぐため設定。
#        save_steps = 10        : モデルを保存するステップ数を指定しています。この例では、10ステップごとにモデルが保存されます。
#        data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
#                                 ここでは、バッチ内のデータをまとめるためのDataCollatorを指定する。
#                                 DataCollatorForLanguageModelingは、言語モデリングタスク（例えば、マスクされた言語モデリングや因果的言語モデリングなど）のためのDataCollator
#                                 mlm=Falseは、マスクされた言語モデリングを行わないことを指定する。

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        # per_device_train_batch_size=4,
        # gradient_accumulation_steps=4,
        # warmup_steps=50,
        # max_steps=500,
        # warmup_steps=5,
        # max_steps=200, # Epoch
        # max_steps=10, # Epoch
        # learning_rate=2e-4,
        # fp16=True,
        logging_steps=1,
        output_dir="./output",
        num_train_epochs = 5,
        warmup_steps = 10,
        weight_decay = 0.1,
        save_steps = 10,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

---

# ファインチューニング実行、推論実行

## 推論（ファインチューニング前） ###

In [ ]:

# モデルに推論させるための入力テキストを設定
input_text = "GPUセンサーコレクションのURIは？"
# input_text = "日本の有名な山は？"
# input_text = "日本の有名な観光地は？"

# トークナイズ
# 入力テキストをトークナイザーでトークナイズし、PyTorchのテンソルに変換して。そのテンソルをGPUまたはCPU（device）に送っている。
input = tokenizer(input_text, return_tensors="pt").to(device)

# モデルによる推論実行
# with torch.no_grad(): ...:
#     推論中に勾配の計算を行わないように指定しています。これにより、推論の速度が向上し、メモリ使用量が削減されます。
#     モデルに入力テキストを与えて推論を行い、その結果を取得しています。max_new_tokens=24は、生成するトークンの最大数を指定しています。
#     pad_token_id=tokenizer.pad_token_idは、生成するトークンが最大数に達した場合に追加するパディングトークンのIDを指定しています。

# モデルによる推論実行
with torch.no_grad():
    output = model.generate(**input, max_new_tokens=24, pad_token_id=tokenizer.pad_token_id, do_sample=True, temperature=0.7)

# 出力テンソルをデコード（文字化）
# skip_special_tokens=Trueは、特殊トークン（例えば、パディングトークンやセパレータートークンなど）をデコード結果から除外することを指定しています。
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# 結果を出力
print(output_text)

## ファインチューニングの実行

In [ ]:
trainer.train()

## 推論実行（ファインチューニング後） ###


In [ ]:
# モデルに推論させるための入力テキストを設定
input_text = "GPUセンサーコレクションのURIは？"

# トークナイズ
input = tokenizer(input_text, return_tensors="pt").to(device)

# モデルによる推論実行
# with torch.no_grad():
#     output = model.generate(**input, max_new_tokens=24, pad_token_id=tokenizer.pad_token_id,)

# モデルによる推論実行
with torch.no_grad():
    output = model.generate(**input, max_new_tokens=24, pad_token_id=tokenizer.pad_token_id, do_sample=True, temperature=0.7)

# 出力テンソルをデコード（文字化）
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# 結果を出力
print(output_text)

In [ ]:
# CSVファイルの内容を確認

df


In [ ]:
# 参考：学習後のモデルを保存
# trainer.save_model("./output/fine_tuned_model")